# Comparison of two cities: Toronto and New York #

### Introduction ###

New York City is the most populous city in the Unites States with a population of over 8 million people. It is also one of the most popular cities in the United States and a centre stage for many cultural events. In the same way Toronto is the most populous city in Canada wit a population of over 2 million people. Toronto is deemed as an international centre of bussiness and culture and is recognized as one of the most multicultural and cosmopolitan cities in the world. 

Two major cities of two major countries. An analysis comparing this two major cities would be of interest for people looking to setle in one of this cities. To have a vibrant and culturally stimulating life.

### Data ###

The data obtained comes from different sources. First, For the analysis, information on the list of neighbourhoods and the coordinates is needed to obtain information on the various venues present in each neibourhood and each city.

Initially, for Toronto, a table with neighbourhoods and postcodes was obtained from wikipedia. The table was cleaned to eliminate rows with 'not assigned' burough and for rows with 'not assigned' neighbours, the name of the burough was used instead.There was the instance of more than one neighbour per postcode. In this case the rows where merged to create one row with one postcode and the name of both neibourhoods separated by a comma.

We obtain the coordinates from http://cocl.us/Geospatial_data . The file contained the postcodes and their respective coordinates. We then merged both tables.

For New York City,

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import json

In [2]:
# getting data and setting beautifulsoup
webseitdata=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(webseitdata,'html.parser')
# create lists to store data
postalCodeList = []
boroughList = []
neighborhoodList = []
#The table is available at class
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if (len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text)
postalCodeList=list(map(lambda x:x.rstrip("\n"),postalCodeList))
boroughList=list(map(lambda x:x.rstrip("\n"),boroughList))
neighborhoodList=list(map(lambda x:x.rstrip("\n"),neighborhoodList))
df=pd.DataFrame({"PostalCode":postalCodeList, "Borough":boroughList,"Neighborhood":neighborhoodList})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Here we process the table to deal with missing values as explained above.

In [3]:
#missing Burough data
df_clean=df[df.Borough!="Not assigned"].reset_index(drop=True)
# dealing with more than one neighbour per postcode
df_clean = df_clean.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
#missing neighbour data
df_clean["Neighborhood"]=df_clean["Neighborhood"].replace("Not assigned",df_clean["Borough"])
df_clean.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df_clean.shape

(103, 3)

We then get the coordinates for each postcode to use later to get venues information.

In [5]:
#get the table with the long and lat coordinates using the table at http://cocl.us/Geospatial_data
can_geo=pd.read_csv('http://cocl.us/Geospatial_data')
can_geo.columns=['PostalCode', 'Latitude', 'Longitude']
# Merge the table of coordinates with the previous table of boroughs and neighbourhoods using the postal code
toronto_df=pd.merge(df_clean,can_geo,how='inner',on="PostalCode")
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


For New York City, we obtained the data of the neighbourhoods and coordinates from https://cocl.us/new_york_dataset . The file is a json file that contains the borough, neibourhoods and coordinates of New York City.

In [6]:
#getting new york data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
new_york = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    new_york = new_york.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
new_york.head()
    


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The Toronto has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)
print('The New York has {} boroughs and {} neighborhoods.'.format(
        len(new_york['Borough'].unique()),
        new_york.shape[0]
    )
)

The Toronto has 10 boroughs and 103 neighborhoods.
The New York has 5 boroughs and 306 neighborhoods.


In [13]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium 

In [15]:
address = 'Toronto,ON'

geolocator = Nominatim(user_agent="ON")
location = geolocator.geocode(address)
latitudet = location.latitude
longitudet = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitudet, longitudet))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values of boroughs and neibourhoods of Toronto
map_toronto = folium.Map(location=[latitudet, longitudet], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [20]:
# create map of new york using latitude and longitude values of boroughs and neibourhoods of Toronto
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(new_york['Latitude'], new_york['Longitude'], new_york['Borough'], new_york['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Using the coordinate information and the Foursquare API, we get the information on the venues for each neighbourhood in both Toronto and New York City.

In [21]:
#Foursquare credentials definition
CLIENT_ID = 'OYQSKVZSJUT4DHCFH0SGXQO5Z1AJ0PSHV0C5N4G3IJDT5BMH' # your Foursquare ID
CLIENT_SECRET = 'KYQ1KTNDYSDJWW2STTLPRLGR21VMH4ZBXYNRZ5TGK2M4TNHT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [22]:
# Fuction that gets the venues within 500 metres from each neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [24]:
newyork_venues = getNearbyVenues(names=new_york['Neighborhood'],
                                   latitudes=new_york['Latitude'],
                                   longitudes=new_york['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

### Methodology

Once we have obtained the neighbourhood and venues data, we can move on analyzing the information. Because the sheer number of the information, we will calculate the frequency of types of venues and Kmeans to cluster the data and compare the clusters between the cities.

### Analysis

In [26]:
print('{} Toronto venues were returned by Foursquare.'.format(toronto_venues.shape[0]))
print('{} New Yorkvenues were returned by Foursquare.'.format(newyork_venues.shape[0]))

2154 Toronto venues were returned by Foursquare.
10097 New Yorkvenues were returned by Foursquare.


In order to use KMeans, we expand the Venue Category column. Using get_dummies, we create new columns with the categories on 'Venues Categories.Then group them by Neighbourhood to get the frequency.  

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
#print(toronto_onehot['Neighborhood'] )
toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]
#print(toronto_onehot['Neighborhood'] )
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Volleyball Court,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Arrochar,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


This is a function to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Select the top ten common venues and ordering them.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
   toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Skating Rink,Clothing Store,Lounge,Latin American Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Skating Rink,Athletics & Sports,Pub,Sandwich Place,Women's Store,Diner,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Pizza Place,Restaurant,Sushi Restaurant,Deli / Bodega,Middle Eastern Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,"Bedford Park, Lawrence Manor East",Restaurant,Sandwich Place,Coffee Shop,Italian Restaurant,Thai Restaurant,Indian Restaurant,Fast Food Restaurant,Japanese Restaurant,Liquor Store,Grocery Store
5,Berczy Park,Coffee Shop,Seafood Restaurant,Bakery,Restaurant,Beer Bar,Cocktail Bar,Farmers Market,Café,Cheese Shop,Eastern European Restaurant
6,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
7,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
8,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Butcher,Auto Workshop
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Coffee Shop,Rental Car Location,Plane,Bar,Boat or Ferry,Harbor / Marina,Airport Gate


In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
newyork_venues_sorted = pd.DataFrame(columns=columns)
newyork_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
   newyork_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

newyork_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Bus Station,Bakery,Intersection,Check Cashing Service,Grocery Store,Gas Station,Breakfast Spot
1,Annadale,Pizza Place,American Restaurant,Pub,Bakery,Pharmacy,Liquor Store,Diner,Restaurant,Train Station,Food
2,Arden Heights,Pizza Place,Pharmacy,Coffee Shop,Bus Stop,Lawyer,Field,Ethiopian Restaurant,Event Service,Event Space,Exhibit
3,Arlington,Bus Stop,Intersection,General Entertainment,Deli / Bodega,Boat or Ferry,Grocery Store,Women's Store,Farmers Market,Field,Fast Food Restaurant
4,Arrochar,Bus Stop,Italian Restaurant,Deli / Bodega,Liquor Store,Mediterranean Restaurant,Middle Eastern Restaurant,Outdoors & Recreation,Sandwich Place,Athletics & Sports,Bagel Shop
5,Arverne,Surf Spot,Metro Station,Sandwich Place,Thai Restaurant,Wine Shop,Burrito Place,Beach,Bus Stop,Playground,Bed & Breakfast
6,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Deli / Bodega,Food Truck,Seafood Restaurant,Bakery,Pizza Place
7,Astoria Heights,Italian Restaurant,Business Service,Gourmet Shop,Hostel,Pizza Place,Bowling Alley,Plaza,Playground,Supermarket,Bakery
8,Auburndale,Italian Restaurant,Mobile Phone Shop,Sushi Restaurant,Toy / Game Store,Pharmacy,Fast Food Restaurant,Train,Miscellaneous Shop,Supermarket,Mattress Store
9,Bath Beach,Chinese Restaurant,Pharmacy,Sushi Restaurant,Donut Shop,Pizza Place,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant,Gas Station,Bank


Using Kmeans to identify 5 clusters within our neighbourhoods.

In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeanst = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeansn = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeansn.labels_[0:10] 

array([4, 4, 4, 3, 3, 4, 4, 4, 4, 4], dtype=int32)

In [34]:
# add clustering labels
toronto_venues_sorted.insert(0, 'ClusterLabels', kmeanst.labels_)
newyork_venues_sorted.insert(0, 'ClusterLabels', kmeansn.labels_)

In [35]:

toronto_merged = toronto_df
toronto_merged = toronto_merged.drop([16,0], axis=0)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Bar,Home Service,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Deli / Bodega
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Electronics Store,Restaurant,Rental Car Location,Mexican Restaurant,Breakfast Spot,Intersection,Medical Center,Bank,Diner,Dim Sum Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Soccer Field,Korean Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Fried Chicken Joint,Gas Station,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Lounge,Hakka Restaurant,Convention Center
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1.0,Playground,Health & Beauty Service,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Hobby Shop,Department Store,Coffee Shop,Discount Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bus Line,Bakery,Park,Bus Station,Soccer Field,Intersection,Ice Cream Shop,Discount Store,Dim Sum Restaurant,Diner
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,1.0,American Restaurant,Skating Rink,Motel,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
10,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,1.0,Indian Restaurant,Pet Store,Thrift / Vintage Store,Brewery,Chinese Restaurant,Light Rail Station,Vietnamese Restaurant,Comic Shop,Colombian Restaurant,Eastern European Restaurant


In [36]:

newyork_merged = new_york
newyork_merged = newyork_merged.drop([16,0], axis=0)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(newyork_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
newyork_merged.dropna(inplace=True)
newyork_merged

,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bronx,Co-op City,40.874294,-73.829939,4.0,Fast Food Restaurant,Bagel Shop,Park,Trail,Discount Store,Pizza Place,Restaurant,Salon / Barbershop,Pharmacy,Bus Station
2,Bronx,Eastchester,40.887556,-73.827806,3.0,Caribbean Restaurant,Bus Station,Deli / Bodega,Diner,Bus Stop,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant,Bowling Alley,Pizza Place
3,Bronx,Fieldston,40.895437,-73.905643,4.0,Music Venue,River,Bus Station,Plaza,Women's Store,Filipino Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
4,Bronx,Riverdale,40.890834,-73.912585,4.0,Bus Station,Medical Supply Store,Park,Bank,Food Truck,Locksmith,Plaza,Gym,Fast Food Restaurant,Event Service
5,Bronx,Kingsbridge,40.881687,-73.902818,4.0,Pizza Place,Bar,Mexican Restaurant,Latin American Restaurant,Sandwich Place,Spanish Restaurant,Donut Shop,Supermarket,Bakery,Pharmacy
6,Manhattan,Marble Hill,40.876551,-73.910660,4.0,Gym,Coffee Shop,Yoga Studio,Supplement Shop,Donut Shop,Pharmacy,Discount Store,Kids Store,Diner,Miscellaneous Shop
7,Bronx,Woodlawn,40.898273,-73.867315,3.0,Pub,Deli / Bodega,Pizza Place,Playground,Food & Drink Shop,Indian Restaurant,Plaza,Park,Bar,Donut Shop
8,Bronx,Norwood,40.877224,-73.879391,4.0,Pizza Place,Chinese Restaurant,Bank,Park,Coffee Shop,Pharmacy,Caribbean Restaurant,Mobile Phone Shop,Mexican Restaurant,Athletics & Sports
9,Bronx,Williamsbridge,40.881039,-73.857446,4.0,Caribbean Restaurant,Bar,Soup Place,Nightclub,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
10,Bronx,Baychester,40.866858,-73.835798,4.0,Donut Shop,Fried Chicken Joint,Discount Store,Sporting Goods Shop,Spanish Restaurant,Supermarket,Fast Food Restaurant,Mexican Restaurant,Men's Store,Mattress Store


In [43]:
# create map
toronto_merged['ClusterLabels']=toronto_merged['ClusterLabels'].astype(int)
mapt_clusters = folium.Map(location=[latitudet, longitudet], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapt_clusters)
       
mapt_clusters

In [44]:
# create map
newyork_merged['ClusterLabels']=newyork_merged['ClusterLabels'].astype(int)
mapn_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapn_clusters)
       
mapn_clusters

- Toronto's Clusters

In [46]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [47]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Bar,Home Service,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Deli / Bodega
2,Scarborough,1,Electronics Store,Restaurant,Rental Car Location,Mexican Restaurant,Breakfast Spot,Intersection,Medical Center,Bank,Diner,Dim Sum Restaurant
3,Scarborough,1,Coffee Shop,Soccer Field,Korean Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
4,Scarborough,1,Fried Chicken Joint,Gas Station,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Lounge,Hakka Restaurant,Convention Center
5,Scarborough,1,Playground,Health & Beauty Service,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
6,Scarborough,1,Hobby Shop,Department Store,Coffee Shop,Discount Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
7,Scarborough,1,Bus Line,Bakery,Park,Bus Station,Soccer Field,Intersection,Ice Cream Shop,Discount Store,Dim Sum Restaurant,Diner
8,Scarborough,1,American Restaurant,Skating Rink,Motel,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
9,Scarborough,1,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
10,Scarborough,1,Indian Restaurant,Pet Store,Thrift / Vintage Store,Brewery,Chinese Restaurant,Light Rail Station,Vietnamese Restaurant,Comic Shop,Colombian Restaurant,Eastern European Restaurant


In [48]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Music Venue,Garden,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dance Studio


In [49]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 3,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3,Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
21,North York,3,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
23,North York,3,Park,Convenience Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
25,North York,3,Park,Food & Drink Shop,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
40,East York,3,Park,Convenience Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
44,Central Toronto,3,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Ethiopian Restaurant
50,Downtown Toronto,3,Park,Playground,Trail,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
74,York,3,Park,Pool,Women's Store,Gluten-free Restaurant,Gift Shop,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
90,Etobicoke,3,Park,Pool,River,College Rec Center,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
91,Etobicoke,3,Park,Deli / Bodega,Baseball Field,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [50]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 4,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,North York,4,Japanese Restaurant,Pizza Place,Bakery,Pub,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
89,Etobicoke,4,Pizza Place,Coffee Shop,Gym,Skating Rink,Athletics & Sports,Pub,Sandwich Place,Women's Store,Diner,Department Store
96,North York,4,Gym,Pizza Place,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
99,Etobicoke,4,Pizza Place,Discount Store,Coffee Shop,Middle Eastern Restaurant,Intersection,Sandwich Place,Chinese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant


- New York City's Clusters

In [51]:
newyork_merged.loc[newyork_merged['ClusterLabels'] == 0,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Clason Point,Park,Convenience Store,Grocery Store,Pool,Bus Stop,Boat or Ferry,South American Restaurant,Women's Store,Field,Event Space
188,Laurelton,Caribbean Restaurant,Park,Train Station,Women's Store,Field,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
192,Somerville,Park,Women's Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
203,Todt Hill,Park,Women's Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [52]:
newyork_merged.loc[newyork_merged['ClusterLabels'] == 1,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
172,Breezy Point,Beach,Trail,Monument / Landmark,Women's Store,Filipino Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
179,Neponsit,Beach,Bar,Financial or Legal Service,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
204,South Beach,Beach,Pier,Deli / Bodega,Bus Stop,Athletics & Sports,Women's Store,Filipino Restaurant,Event Space,Exhibit,Factory
302,Hammels,Beach,Shoe Store,Diner,Food Truck,Bus Station,Bus Stop,Fast Food Restaurant,Gym / Fitness Center,Dog Run,Fried Chicken Joint


In [53]:
newyork_merged.loc[newyork_merged['ClusterLabels'] == 2,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
183,Jamaica Estates,Bus Station,Dog Run,Women's Store,Financial or Legal Service,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
202,Grymes Hill,Dog Run,Women's Store,Filipino Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [55]:
newyork_merged.loc[newyork_merged['ClusterLabels'] == 3,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Eastchester,Caribbean Restaurant,Bus Station,Deli / Bodega,Diner,Bus Stop,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant,Bowling Alley,Pizza Place
7,Woodlawn,Pub,Deli / Bodega,Pizza Place,Playground,Food & Drink Shop,Indian Restaurant,Plaza,Park,Bar,Donut Shop
23,Longwood,Deli / Bodega,Grocery Store,Diner,Fast Food Restaurant,Sandwich Place,Latin American Restaurant,Donut Shop,Women's Store,Field,Event Service
28,Throgs Neck,Italian Restaurant,Pizza Place,Sports Bar,Mobile Phone Shop,Baseball Field,Coffee Shop,Asian Restaurant,Bar,American Restaurant,Deli / Bodega
32,Van Nest,Deli / Bodega,Pizza Place,Bus Station,Bakery,Supermarket,Hookah Bar,Coffee Shop,Middle Eastern Restaurant,College Gym,Chinese Restaurant
39,Edgewater Park,Italian Restaurant,Pizza Place,Coffee Shop,Deli / Bodega,Bar,Park,Liquor Store,Donut Shop,Asian Restaurant,Sports Bar
41,Olinville,Caribbean Restaurant,Furniture / Home Store,Basketball Court,Deli / Bodega,Supermarket,Metro Station,Food,Filipino Restaurant,Exhibit,Factory
43,Concourse,Deli / Bodega,Grocery Store,Chinese Restaurant,Pizza Place,Fried Chicken Joint,Clothing Store,Bakery,Caribbean Restaurant,Pharmacy,Fast Food Restaurant
74,Canarsie,Chinese Restaurant,Deli / Bodega,Bus Line,Food,Gym,Caribbean Restaurant,Home Service,Asian Restaurant,Women's Store,Field
83,Marine Park,Baseball Field,Park,Pizza Place,Gym,Athletics & Sports,Ice Cream Shop,Chinese Restaurant,Deli / Bodega,Basketball Court,Soccer Field


In [56]:
newyork_merged.loc[newyork_merged['ClusterLabels'] == 4,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Fast Food Restaurant,Bagel Shop,Park,Trail,Discount Store,Pizza Place,Restaurant,Salon / Barbershop,Pharmacy,Bus Station
3,Fieldston,Music Venue,River,Bus Station,Plaza,Women's Store,Filipino Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
4,Riverdale,Bus Station,Medical Supply Store,Park,Bank,Food Truck,Locksmith,Plaza,Gym,Fast Food Restaurant,Event Service
5,Kingsbridge,Pizza Place,Bar,Mexican Restaurant,Latin American Restaurant,Sandwich Place,Spanish Restaurant,Donut Shop,Supermarket,Bakery,Pharmacy
6,Marble Hill,Gym,Coffee Shop,Yoga Studio,Supplement Shop,Donut Shop,Pharmacy,Discount Store,Kids Store,Diner,Miscellaneous Shop
8,Norwood,Pizza Place,Chinese Restaurant,Bank,Park,Coffee Shop,Pharmacy,Caribbean Restaurant,Mobile Phone Shop,Mexican Restaurant,Athletics & Sports
9,Williamsbridge,Caribbean Restaurant,Bar,Soup Place,Nightclub,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
10,Baychester,Donut Shop,Fried Chicken Joint,Discount Store,Sporting Goods Shop,Spanish Restaurant,Supermarket,Fast Food Restaurant,Mexican Restaurant,Men's Store,Mattress Store
11,Pelham Parkway,Deli / Bodega,Italian Restaurant,Frozen Yogurt Shop,Pizza Place,Spa,Bakery,Sushi Restaurant,Track,Coffee Shop,Liquor Store
12,City Island,Thrift / Vintage Store,Seafood Restaurant,Harbor / Marina,Smoke Shop,Bank,Baseball Field,Pharmacy,Music Venue,History Museum,Boat or Ferry


### Results and discussion

Segmenting both cities using KMeans yielded 5 klusters per city. 

The clusters for Toronto:

- Cluster 1

There are no elements in this cluster
- Cluster 2

It is the largest cluster with 81 neighbourhoods out of 103. It shows a variety of restaurants from a variety of ethnicities and caffes.
- Cluster 3

There are not enough neighbourhood to form an opinion. Although similarly to cluster 1, there are a lot event related venues. Caffes seem to be very popular in the neighbourhoods of this region.
- Cluster 4 

The elements of this cluster have a major frequency of Park areas and Discount stores.
- Cluster 5 

The neighbourhoods of this cluster have all, at first or second place, Pizza place. They also have in similar women's store and distribution centre. 

The clusters for New York City :

- Cluster 1

Every neighbourhood boots in its top ten:  parks, event services, event space, and exhibits.
- Cluster 2

They all seem to be situated along beaches.
- Cluster 3

There are not enough neighbourhood to form an opinion. Although similarly to cluster 1, there are a lot event related venues.
- Cluster 4 

This cluster has a preponderance of Deli/Bodegas. It also shows a large variety of restaurants from many different ethnicities. A statement of the multicultural proprerty of New York City.
- Cluster 5 

It is the largest cluster with 249 neighbourhoods out of 306. It shows a variety of restaurants and leisure venues.

### Conclussion

Although both cities are the most populous cities of their country, New York City has 4 times more inhabitants that its counterpart, Toronto. This is reflected also in the number of neighbourhoods: 103 and 306 for Toronto and New York respectively. But it is in the venues where the difference is more seen; the 2154 of Toronto venues agains the 10097 of New York City. The number of venues in New York City is 5 times that of tge venues in Toronto.

The segmentation was very looped with one cluster having about 80% of the neibouroods and one or two clusters having 0-2 neighbourhoods. But some charateristics are still observable. Both cities show their multicultural cusine and the importance of leisure activities. The importance of the culture of the Deli/Bodegas in New York City is seen in cluster 4. And while there is a variety of cusines, Pizza places and caffes seem to be the most important in Toronto as seen in cluster 2 and 5.

We have outlined some similarities and differences of both this cities based on the types and frequency of venues. For people deciding to transfer, there are still others deciding factors as 